Once you have a scraped tumblr corpus you want to fine-tune GPT-2 on, this preps the data for training.

To get a corpus, use a tumblr scraper such as https://gist.github.com/doersino/7e3e5db591e42bf543e1

After prepping the data, use `gpt-2/train.py` to finetune.

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config Completer.use_jedi = False

In [24]:
import os

import bs4
from bs4 import BeautifulSoup

In [25]:
def lprint(s, prefix=""):
    print(f"{prefix}{s}", end=f"\n\n{prefix}---------\n\n")

In [26]:
POSTS_DIR = ""  # wherever the posts are, as .html files

In [27]:
Q_CHAR = "会"
A_CHAR = "域"
T_CHAR = "职"

UNAME_CHAR = "友"
ORIG_POST_CHAR = "翰"

EOT_FULL = "<|endoftext|>"

In [28]:
from reblogs_v5 import *

In [29]:
# a lot of hardcoded magic numbers and stuff here -- i tuned a lot of stuff here "to my taste"
# eg excluding reblog chains that have too much writing by others and not much by me

def screen_for_inclusion(processed, post_metadata):
    body, _, tags = processed.partition(T_CHAR)
    
    if post_metadata["is_quotes"] == True:
        return False, 0, 0
    
    if ORIG_POST_CHAR in body:
        other_body = ""
        me_body = body.split(ORIG_POST_CHAR)[1]
    elif A_CHAR in body:
        me_body, other_body = "", ""
        in_other = True
        for char in body:
            if char == Q_CHAR:
                in_other = True
            if char == A_CHAR:
                in_other = False
            if in_other:
                other_body += char
            else:
                me_body += char
    else:
        return False, 0, 0
    
    if len(me_body) < 10:
        return False, len(me_body.split()), len(other_body.split())
    if (len(other_body) / len(me_body)) > 2 and len(me_body.split()) < 250:
        print(f"rejecting other_body {len(other_body.split())} words, me_body {len(me_body.split())} words")
        return False, len(me_body.split()), len(other_body.split())
    
    return True, len(me_body.split()), len(other_body.split())

In [30]:
from collections import Counter

def get_all_posts(posts_dir=POSTS_DIR, limit=None):
    posts = []
    post_fns = []
    meta_counts = Counter()
    all_meta_counts = Counter()
            
    all_fns = os.listdir(posts_dir)
    for ix, fn in enumerate(all_fns):
        if not fn.endswith(".html"):
            continue
    
        with open(os.path.join(posts_dir, fn), "r") as f:
            soup = BeautifulSoup(f)

        try:
            processed, post_metadata = process_post(soup, uname_config="frank_v5_train", debug=False)
        except Exception as e:
            print(f"hit {e} on {fn}")
            continue
            
        for key in sorted(post_metadata.keys()):
            all_meta_counts[key] += post_metadata[key]
           
        passed_screen, words_me, words_other = screen_for_inclusion(processed, post_metadata)
        
        all_meta_counts["words_me"] += words_me
        all_meta_counts["words_other"] += words_other
        
        if passed_screen:
            for key in sorted(post_metadata.keys()):
                meta_counts[key] += post_metadata[key]
                
            meta_counts["words_me"] += words_me
            meta_counts["words_other"] += words_other
                
            posts.append(processed)
            post_fns.append(fn)
         
        if ix % 500 == 0:
            print(f"{ix}/{len(all_fns)}\n")
            for k in meta_counts.keys():
                print(f"incl_meta_counts[{k}]:\t{meta_counts[k]}\nall__meta_counts[{k}]:\t{all_meta_counts[k]}\n")
            print()
            
        if limit is not None:
            if ix >= limit:
                break
            
    return posts, meta_counts, post_fns

In [1]:
posts, meta_counts, post_fns = get_all_posts()

In [2]:
total_me = meta_counts["words_me"]
total_other = meta_counts["words_other"]

print(f"{total_me//1000}K words from me")
print(f"{total_other//1000}K words from others")
print()
print(f"{total_me / (total_me + total_other):.1%} me")

In [3]:
# review examples
from textwrap import fill

subset_review = [p for p in posts if not p.startswith("翰")]

for p in np.random.choice(subset_review, 10):
    print(p)
    print("\n\n" + 20*"~~~~~" + "\n\n")

In [45]:
posts_string = "".join(posts)

In [46]:
TRAIN_DATA_PATH = ""  # fill in
with open(TRAIN_DATA_PATH, "w", encoding="utf-8") as f:
    f.write(posts_string)